### 1. Install requiered packages

In [ ]:
!nvidia-smi

Wed Aug  7 18:23:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!apt-get install -qq pciutils
!apt-get install -qq lshw

Selecting previously unselected package pci.ids.
(Reading database ... 123598 files and directories currently installed.)
Preparing to unpack .../pci.ids_0.0~2022.01.22-1_all.deb ...
Unpacking pci.ids (0.0~2022.01.22-1) ...
Selecting previously unselected package libpci3:amd64.
Preparing to unpack .../libpci3_1%3a3.7.0-6_amd64.deb ...
Unpacking libpci3:amd64 (1:3.7.0-6) ...
Selecting previously unselected package pciutils.
Preparing to unpack .../pciutils_1%3a3.7.0-6_amd64.deb ...
Unpacking pciutils (1:3.7.0-6) ...
Setting up pci.ids (0.0~2022.01.22-1) ...
Setting up libpci3:amd64 (1:3.7.0-6) ...
Setting up pciutils (1:3.7.0-6) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.4) ...
/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_opencl.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_level_zero.so.0 is not a symbolic link

/sb

In [ ]:
!pip install tqdm
!pip install transformers
!pip install langchain
!pip install langchain-community
!pip install pinecone-client
!pip install langchain-pinecone
!pip install sentence-transformers
!pip install langchain_ollama
!pip install colab-xterm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 10.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cu

###2. Connect to a LLM

In [ ]:
%load_ext colabxterm
!curl -fsSL https://ollama.com/install.sh | sh
!ollama serve > ollama.log 2>&1 &
!sleep 10
!ollama pull llama3   # you can use llama3, phi3... any model available in https://ollama.com/library

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1e

###3. Import required tools

#### Packages

In [ ]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain_pinecone import Pinecone
from langchain_community.llms import Ollama
from pinecone import Vector
from tqdm import tqdm
import subprocess
import pinecone
import time
import os
import warnings

warnings.filterwarnings("ignore")

####Embedding model

In [ ]:
# Define an embedding model
embedding_model_name = 'sentence-transformers/all-mpnet-base-v2'
embedding_model = HuggingFaceBgeEmbeddings(model_name=embedding_model_name)

###3. App

####utils

In [ ]:
# DATABASE CREATION FUNCTIONS
# Function to upload and process pdf documents into a vectorized Database
def process_and_upsert_documents(docs_path, pnc_index, embedding_model):
  # intialize text splitter
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

  # list of documents names
  docs_names = os.listdir(docs_path)

  # process documents
  for name in docs_names:
      if name.endswith('.pdf'):
          # create file paths
          file_path = os.path.join(docs_path, name)

          try:
              # upload pdf documents
              loader = PyMuPDFLoader(file_path)
              document = loader.load()

              # chunk documents
              chunks = text_splitter.split_documents(document)

              # create embedding vectors based on chunks
              vectors = []
              for i, chunk in enumerate(chunks):
                  vector_id = f"{name}-{i}"
                  vector_values = embedding_model.embed_documents([chunk.page_content])[0]
                  vector = Vector(
                      id=vector_id,
                      values=vector_values,
                      metadata=chunk.metadata
                  )
                  vectors.append(vector)

              # upsert vectors into the databse
              batch_size = 100
              total_vectors = len(vectors)
              for i in tqdm(range(0, total_vectors, batch_size), desc=f"Upserting {name}"):
                  batch = vectors[i:i+batch_size]
                  pnc_index.upsert(
                      vectors=batch,
                      namespace=name,
                      show_progress=False
                  )

              print(f'Document {name} was succesfully upload!')

          except Exception as e:
              print(f'Error in processing file: {name}: {e}')
  print("Your Vectorized Data Base was succesfully created!")



# CHAT LOGIC FUNCTIONS
# Function to create memory in chat
def create_memory():
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    return memory


# Function to create conversation chain
def create_qa(llm, vectorstore, memory):
  qa = ConversationalRetrievalChain.from_llm(
  llm=llm,
  retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
  memory=memory
)

  return qa



# CHAT DISPLAY FUNCTIONS
# header
def header():
    print("¡Hello, I am Boto, your personal AI-Assistant! (Type 'exit' to quit)")


# user
def user_input():
    return input("You: ")


# bot
def bot_response(response):
    print("🤖 Assistant:", response)

# exit message
def if_exit(user_input):
    if user_input.lower() == "exit":
        print("Goodbye!")
        return True



# APP DISPLAY FUNCTIONS
# Function to give a welcome message
def welcome():
    print("Welcome to your personal AI-Assistant app🤖")


# Function to display an app menu
def options():
    options = ['Chat with Boto', 'Create a new DataBase', 'Exit']

    for number, option in enumerate(options, start=1):
        print(f'{number}. {option}')

    answer = None
    while answer is None:
        answer = input('What would you like to do?')
        try:
            answer = int(answer)
        except ValueError as e:
            answer = None
            print(f"{e}: Type (1) or (2) to select an option or (3) to exit")

        if answer == 1 or answer == 2 or answer == 3:
            os.system('cls')
            return answer
        else:
            answer = None
            print("Type (1) or (2) to select an option or (3) to exit")


####database

In [ ]:
def create_DB(API_KEY_PINECONE, INDEX_NAME_PINECONE):
    # get documents path
    current_dir = os.getcwd()
    docs_path = os.path.join(current_dir, "documents")

    # Initialize pinecone DB
    try:
        pncDB = pinecone.Pinecone(api_key=API_KEY_PINECONE)
        index = pncDB.Index(INDEX_NAME_PINECONE)
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

    process_and_upsert_documents(docs_path=docs_path,pnc_index=index)


####chat

In [ ]:
def chat(API_KEY_PINECONE, INDEX_NAME_PINECONE, embedding_model):
    # Initialize pinecone DB
    try:
        pncDB = pinecone.Pinecone(api_key=API_KEY_PINECONE)
        index = pncDB.Index(INDEX_NAME_PINECONE)
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

    # Initialize phi3 llm
    system_message = "You are an AI assistant that helps to write my thesis. Your name is Boto and always respond only in English lenguage."
    llm = Ollama(model="llama3", temperature=0.7, system=system_message)

    # Initialize vector store
    vectorstore = Pinecone(index, embedding_model, "text")

    # Create chat memory
    memory = create_memory()

    # Create conversation chain in chat
    qa = create_qa(llm=llm, vectorstore=vectorstore, memory=memory)


    # Display header
    header()

    # Chat display
    while True:
        # user input
        input = user_input()

        # validate if user wants to exit
        exit = if_exit(user_input=input)
        if exit:
            break

        # get a bot responser
        result = qa({"question": input})
        response = result['answer']

        # display bot response
        bot_response(response=response)

        """
        if 'source_documents' in result:
            print("\nSources:")
            for doc in result['source_documents']:
                print(doc.page_content[:100] + "...")
        """

        print()


####app

In [ ]:
def run_app(API_KEY_PINECONE, INDEX_NAME_PINECONE, embedding_model):
    # welcome message
    welcome()

    # app menu manager logic
    option = None
    while option is None:
        # display options menu
        option = options()

        if option == 1:
            chat(API_KEY_PINECONE, INDEX_NAME_PINECONE, embedding_model)
        elif option == 2:
            create_DB(API_KEY_PINECONE, INDEX_NAME_PINECONE)
        else:
            break

        option = None
        os.system('cls')


###4. Run

In [ ]:
# Configure hugging face
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "<Introduce your HuggingFace API Token> "

# Connect to a pinecone DB
API_KEY_PINECONE ="<Introduce your pinecone API Key>"
INDEX_NAME_PINECONE = "<Introduce your pinecone index name>"

In [ ]:
# Run chatbot app
run_app(API_KEY_PINECONE, INDEX_NAME_PINECONE, embedding_model)

Welcome to your personal AI-Assistant app🤖
1. Chat with Boto
2. Create a new DataBase
3. Exit
What would you like to do?1
¡Hello, I am Boto, your personal AI-Assistant! (Type 'exit' to quit)
You: Hello Boto!
🤖 Assistant: Hello! I'm Boto, your AI assistant for thesis-related matters. How can I assist you today? What's on your mind, and how can I help?

You: Boto, can you explain me in a few words, what is a metabolic genome-scale model?
🤖 Assistant: I'd be happy to help you with that.

A metabolic genome-scale model, also known as a metabolic network model or flux balance analysis (FBA) model, is a computational representation of an organism's metabolism at a genomic scale. It integrates genomic information on genes and their corresponding enzymes involved in various metabolic pathways, including gene regulation, enzyme kinetics, and reaction stoichiometry.

In the context of your thesis, a metabolic genome-scale model can be used to:

1. **Predict metabolic capabilities**: By modeling 